In [2]:
from euriai.langchain import create_chat_model

chat_model = create_chat_model(
    api_key="euri-2a913306ad38ca08872c440188785aa42c4f1cc0632c45b431b90cc0ea398992",
    model="gpt-4.1-nano",
    temperature=0.7
)

response = chat_model.invoke("What is artificial intelligence?")
print(response.content)


Artificial intelligence (AI) refers to the development of computer systems and software that can perform tasks typically requiring human intelligence. These tasks include understanding language, recognizing images, making decisions, solving problems, learning from experience, and adapting to new situations. AI can be classified into narrow AI, which is designed for specific tasks (like virtual assistants or recommendation systems), and general AI, which would have the ability to perform any intellectual task a human can do—though such general AI remains a theoretical goal at this stage. Overall, AI aims to create machines that can analyze data, learn from it, and act accordingly to assist or augment human activities.


In [2]:
pip install euriai

In [15]:
pip install  --upgrade euriai

In [5]:
import pydantic
print(pydantic.__version__)

2.11.7


In [7]:
pip install -U langchain langchain-core langchain-community langchain-openai


  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_community-0.3.29-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_text_splitters-0.3.11-py3-none-any.whl.metadata (1.8 kB)
  Using cached sqlalchemy-2.0.43-cp310-cp310-win_amd64.whl.metadata (9.8 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached greenlet-3.2.4-cp310-cp310-win_amd64.whl.metadata (4.2 kB)
  Using cached aiohttp-3.12.15-cp310-cp310-win_amd64.whl.metadata (7.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.met

In [3]:
import ast,re,math,os,sys
os.environ["euri-api-key"] ="euri-2a913306ad38ca08872c440188785aa42c4f1cc0632c45b431b90cc0ea398992"
from euriai.langchain import create_chat_model
from langchain.agents import create_react_agent,AgentExecutor
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate,MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableLambda,RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory ,InMemoryChatMessageHistory as ChatMessageHistory
from langchain_core.tools import BaseTool,tool
from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryMemory,
    CombinedMemory,
    ReadOnlySharedMemory,
    ConversationEntityMemory

)


In [10]:
llm_default = create_chat_model(
    api_key="euri-2a913306ad38ca08872c440188785aa42c4f1cc0632c45b431b90cc0ea398992",
    model="gpt-4.1-nano",
    temperature=0.2
)


In [4]:
@tool("calculator",return_direct=True)
def calculator(expression: str) -> str:
    """Evaluate a numeric math expression . supprts + , -,*,/,**,parentheses and all kind of mathamatical functions"""
    allowed_nodes =(
        ast.Expression,
        ast.UnaryOp,
        ast.unaryop,
        ast.BinOp,
        ast.operator,
        ast.Num,
        ast.Load,
        ast.pow,
        ast.FunctionDef,
        ast.Module,
        ast.Expr,
        ast.Call,
        ast.Name,
        ast.arguments,
        ast.args,
        ast.Constant
    )

    allowed_names = {k:v for k,v in vars(math).items() if not k.startswith("_")}
    allowed_names.update({"abs": abs, "round": round,"min": min,"max": max})
    node = ast.parse(expression, mode="eval")
    
    
    for n in ast.walk(node):
        if not isinstance(n, allowed_nodes):
            raise ValueError(f"Expression contains invalid node {type(n)}")
        if isinstance(n, ast.Name) and n.id not in allowed_names:
            raise ValueError(f"Expression contains invalid name {n.id}")
    code = compile(node, "<string>", "eval")
    return str(eval(code, {"__builtins__": {}}, allowed_names))


In [5]:
tools_math = [calculator]

In [12]:
react_prompt_math = ChatPromptTemplate.from_messages([(
    "system","you are a precise math assistant , you can use thse tools:\n{tools}\n"
    "when you are going to use this tools follw these instruction exactly the same format:\n"
    "Questions :......\nthought ...\nAction by using one of the tools access that you have [{tool_names}]\n "
    "alwasy finish with final give me a numeric answer to the question"),
    ("human","questions: {input}\n{agent_scratchpad}")
])



In [32]:
my_agent=create_react_agent(
    llm=llm_default,
    prompt=react_prompt_math,
    tools=tools_math
)


In [14]:
memory_math = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    input_key="input",
    output_key="output",
)


C:\Users\Md Saif Akhtar\AppData\Local\Temp\ipykernel_18144\2855859704.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory_math = ConversationBufferMemory(


In [33]:
math_executor = AgentExecutor(
    agent = my_agent,
    tools=tools_math,
    memory=memory_math,
    verbose=True,
    handle_parsing_errors=
        (
            "you did not follow the instruction that i have given you to use the tools"
        )
)



In [17]:
@tool("kb_search", return_direct=True)
def kb_search(query: str) -> str:
    """a mock function to search from a knowledge base"""
    knowledge_base = {
        "what is the capital of france": "The capital of France is Paris.",
        "who is the president of the united states": "The president of the United States is Joe Biden.",
        "what is the largest mammal": "The largest mammal is the blue whale.",
    }
    return knowledge_base.get(query.lower(), "I don't know the answer to that question.")


In [18]:
tools_kb = [kb_search]


In [20]:
react_prompt_kb = ChatPromptTemplate.from_messages([
    ("system","you are a helpful assistant that can answer question based on your knowledge base and you can use the following tools:\n{tools}\n"
    "when you are going to use this tools follw these instruction exactly the same format:\n"
    "Questions :......\nthought ...\nAction by using one of the tools access that you have [{tool_names}]\n "
    "alwasy finish with final give me a numeric answer to the question"),
    ("human","questions: {input}\n{agent_scratchpad}")
])


In [23]:
mem_kb = ConversationBufferWindowMemory(
    memory_key="chat_history",
    return_messages=True,
    input_key="input",
    output_key="output",
    k=5
)


C:\Users\Md Saif Akhtar\AppData\Local\Temp\ipykernel_18144\2885531510.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  mem_kb = ConversationBufferWindowMemory(


In [38]:
kb_executor = AgentExecutor(
    agent=kb_agent,
    tools=tools_kb,
    memory=mem_kb,
    verbose=True,
    handle_parsing_errors=(
        "you did not follow the instruction that i have given you to use the kb  tools"
    ),
)


In [37]:
kb_agent=create_react_agent(
    llm=llm_default,
    prompt=react_prompt_math,
    tools=tools_kb
)


In [1]:
router_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a router. Read the user message and output exactly one token:\n"
     "- MATH: if it asks for calculations.\n"
     "- KB: otherwise (tech education, RAG, agents, general know-how).\n"
     "Output only MATH or KB."),
    ("human", "{input}")
])

NameError: name 'ChatPromptTemplate' is not defined

In [40]:
def _dispatcher(input:dict):
    """it will take new inputs from human and it will also attach hitory from memory """
    user_msg = inputs[input]
    choice = router_chain.invoke({"input": user_msg}).strip().upper()
    if choice == "MATH":
        return math_exector.invoke({"input": user_msg})
    elif choice == "KB":
        return kb_executor.invoke({"input": user_msg})
    else:
        return "I can only answer math and kb related questions"


In [41]:
dispatcher = RunnableLambda(_dispatcher)

In [42]:
def _get_history(session_id:str):
    if session_id not in _sessions:
        _sessions[session_id] = ChatMessageHistory()
    return _sessions[session_id]


In [43]:
orchestrator = RunnableWithMessageHistory(
    runnable=dispatcher,
    get_session_history=_get_history,
    input_key="input",
    history_messages_key="history"
)
